# Query Rightmove with LangChain | OpenAI | Python | bs4

In [49]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [51]:
import statistics
import requests
from bs4 import BeautifulSoup

In [53]:
url='https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E116&maxBedrooms=3&minBedrooms=3&maxPrice=325000&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords='
url_me='https://www.rightmove.co.uk/property-for-sale/find.html?minBedrooms=2&keywords=&sortType=2&minPrice=250000&viewType=LIST&channel=BUY&index=0&radius=0.0&locationIdentifier=USERDEFINEDAREA%5E%7B%22id%22%3A8289123%7D'

## The Tools

In [156]:
# USE THIS ONE
def get_median_price(url=url) -> str:
    headers = { 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0' }
    response = requests.get(url, headers=headers)

    # Scrape the content

    soup = BeautifulSoup(response.text, "html.parser")

    props = soup.find_all("div", class_="l-searchResult is-list")

    location_l = []
    address_l = []
    price_l = []
    for i in range(len(props)):
        prop = props[i]
        location = (
            prop.find("address", itemprop='address')
            .get_text()
            .replace(",","")
            .replace("\n","")
            .strip("£")
            .strip()
        )
        location_l.append(location)

        address = (
            prop.find("h2", class_="propertyCard-title")
            .get_text()
            .replace(",","")
            .replace("\n","")
            .strip("£")
            .strip()
        )
        address_l.append(address)

        price = (
            prop.find("div", class_="propertyCard-priceValue")
            .get_text()
            .replace(",","")
            .strip("£")
            .strip()
        )
        price_l.append(price)
    return location_l, address_l, price_l

['650000',
 '60000000',
 '45000000',
 '45000000',
 '43700000',
 '35000000',
 '35000000',
 '35000000',
 '35000000',
 '35000000',
 '32000000',
 '32000000',
 '32000000',
 '32000000',
 '30000000',
 '30000000',
 '29975000',
 '29950000',
 '29950000',
 '29700000',
 '29700000',
 '29500000',
 '29500000',
 '28000000',
 '28000000']

In [ ]:
location_l, address_l, price_l = get_median_price(url_me)

In [157]:
data = {
    'Location': location_l,
    'Address': address_l,
    'Price': price_l
}
df = pd.DataFrame(data)
# Clean and convert price to a numeric type for BigQuery
df['Price'] = df['Price'].str.extract('(\d+,\d+|\d+)').replace(",", "", regex=True).astype(float)

# Save the DataFrame to a CSV file
csv_file = 'properties.csv'
df.to_csv(csv_file, index=False)

print(f"Data saved to {csv_file}")

Data saved to properties.csv
